In [ ]:
!pip install transformers accelerate sentencepiece


### LLM 학습에 편하게 데이터 형식 변경 및 JSONL 로 변환

In [30]:
import pandas as pd
import json

# CSV 파일 읽기
data = pd.read_csv('/data/ephemeral/jung/noisy_clean_pairs.csv', sep=',', encoding='utf-8', engine='python')

# JSONL로 저장
with open('/data/ephemeral/jung/noisy_clean_pairs.jsonl', 'w', encoding='utf-8') as f:
    for _, row in data.iterrows():
        json.dump(row.to_dict(), f, ensure_ascii=False)
        f.write('\n')



In [32]:
import json

input_file = '/data/ephemeral/jung/noisy_clean_pairs.jsonl'
output_file = '/data/ephemeral/jung/clean_noisy_clean_pairs.jsonl'

with open(input_file, 'r', encoding='utf-8') as infile, open(output_file, 'w', encoding='utf-8') as outfile:
    for line in infile:
        try:
            data = json.loads(line)
            # `noisy_text`와 `text`가 문자열이 아닌 경우 문자열로 변환
            data["noisy_text"] = str(data["noisy_text"]) if not isinstance(data["noisy_text"], str) else data["noisy_text"]
            data["text"] = str(data["text"]) if not isinstance(data["text"], str) else data["text"]
            # 수정된 데이터를 JSONL 포맷으로 출력
            outfile.write(json.dumps(data, ensure_ascii=False) + '\n')
        except json.JSONDecodeError as e:
            print(f"Error parsing line: {e}")


In [ ]:
# llm 학습 전 메모리 초기화
import torch
torch.cuda.empty_cache()

### 노이즈 데이터 복원 LLM 파인튜닝
### 해당 코드는 중간에 오류로 인하여 코랩에서 돌렸상태 => 그래서 현재 환경에서 작동할지는 조금 애매함

In [ ]:
import torch
from transformers import AutoModelForCausalLM, Trainer, TrainingArguments, AutoTokenizer
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, TaskType

# JSONL 데이터셋 로드
dataset = load_dataset('json', data_files='/clean_noisy_clean_pairs.jsonl', split='train')

# 모델 및 토크나이저 설정
model_name = "sh2orc/Llama-3.1-Korean-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)

# pad_token 설정
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# 모델 로드 (8-bit 양자화 적용)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True,  # 8-bit 양자화
    device_map='auto'
)

# LoRA 구성 설정
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=16,  # 랭크 크기 => 메모리와 성능의 트레이드오프 가 되므로 설정 조금 주의
    lora_alpha=32,
    lora_dropout=0.05
)

# PEFT 모델 생성
model = get_peft_model(model, lora_config)

# GPU 캐시 비우기
torch.cuda.empty_cache()

# 전처리 함수 정의 (변경 없음)
def preprocess_function(examples):
    prompts = [f"### 명령어: 노이즈 텍스트 복원\n\n입력: {noisy}\n\n출력:" for noisy in examples["noisy_text"]]
    targets = [t + tokenizer.eos_token for t in examples["text"]]
    full_texts = [p + " " + t for p, t in zip(prompts, targets)]

    # 전체 텍스트 토큰화
    tokenized_full = tokenizer(full_texts, max_length=216, truncation=True, padding="max_length")

    # 프롬프트 토큰화 (레이블 마스킹을 위해)
    tokenized_prompts = tokenizer(prompts, max_length=216, truncation=True, padding="max_length", add_special_tokens=False)

    labels = []
    for i in range(len(tokenized_full['input_ids'])):
        input_ids = tokenized_full['input_ids'][i]
        label_ids = input_ids.copy()

        # 프롬프트 길이 계산 (패딩 토큰 제외)
        prompt_length = sum([1 for id in tokenized_prompts['input_ids'][i] if id != tokenizer.pad_token_id])

        # 입력 부분 마스킹
        label_ids[:prompt_length] = [-100] * prompt_length

        labels.append(label_ids)

    tokenized_full['labels'] = labels
    return tokenized_full

# 데이터셋 전처리
tokenized_datasets = dataset.map(preprocess_function, batched=True, remove_columns=dataset.column_names)

# 학습 설정
training_args = TrainingArguments(
    output_dir="./results",
    report_to="none",
    evaluation_strategy="no",
    learning_rate=1e-4,  # LoRA 사용 시 학습률을 높일 수 있음
    per_device_train_batch_size=4,  # 배치 크기 증가 가능
    num_train_epochs=3,
    save_total_limit=1,
    save_steps=500,
    fp16=True,  # 혼합 정밀도
    logging_steps=50,
    push_to_hub=True,
    hub_model_id="Toastmachine/LLama3.1_ASCII_noise_filter",
    hub_private_repo=True,
)

# Trainer 초기화
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
)

# 학습 실행
trainer.train()

# 모델 및 토크나이저 저장
trainer.push_to_hub()
tokenizer.push_to_hub("Toastmachine/LLama3.1_ASCII_noise_filter")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Epoch,Training Loss,Validation Loss


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.47 GiB. GPU 0 has a total capacty of 31.74 GiB of which 1.16 GiB is free. Process 76790 has 30.58 GiB memory in use. Of the allocated memory 24.79 GiB is allocated by PyTorch, and 5.41 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
# 노이즈 입력 예시
noisy_input = 'u노트= 주말 2y만대 개s…|장은 불O 9\n금 얼룩'

# 복원된 출력 생성
clean_output = generate_text(noisy_input)

print(f"노이즈 입력: {noisy_input}")
print(f"복원된 출력: {clean_output}")
